In [23]:
import pandas as pd

* All data obtained from data.sandiego.gov

* Data collected according to requirements set forth in Government Code section 12525.5 that was enacted as a result of the Racial and Identity Profiling Act of 2015 (AB 953), also known as RIPA. 

* Data collection began on July 1, 2018, details all stopped conducted by San Diego Police Department

* Includes (data.sandiego.gov, 2023): 
    * Stop details including stop date, time, duration and location, and other details about the person(s) stopped 
    * The primary reason the person(s) was stopped
    * The gender of person(s) stopped, as perceived by the officer conducting the stop
    * The race of persons stopped, as perceived by the officer conducting the stop
    * Whether person(s) stopped displayed signs of deafness, speech impairment, blindness, a mental health condition, intellectual or developmental disability, or another disability categorized as “other"
    * Actions an officer took toward person(s) stopped
    * The basis for any search conducted as part of stop 
    * Contraband or evidence found
    * Basis for seizing property
    * Type of property seized
    * The outcome(s) of a stop 

* Each stop is identified by a unique `stop_id` field and the person(s) stopped is identified by the `pid` field. These fields will be used to join all data files into one all encompassing DataFrame



In [24]:
# read in data
df1 = pd.read_csv('datasd_ripa/ripa_stops_datasd.csv', low_memory=False)
df2 = pd.read_csv('datasd_ripa/ripa_stop_reason_datasd.csv', low_memory=False)
df3 = pd.read_csv('datasd_ripa/ripa_gender_datasd.csv')
df4 = pd.read_csv('datasd_ripa/ripa_race_datasd.csv')
df5 = pd.read_csv('datasd_ripa/ripa_disability_datasd.csv')
df6 = pd.read_csv('datasd_ripa/ripa_actions_taken_datasd.csv')
df7 = pd.read_csv('datasd_ripa/ripa_search_basis_datasd.csv')
df8 = pd.read_csv('datasd_ripa/ripa_contraband_evid_datasd.csv')
df9 = pd.read_csv('datasd_ripa/ripa_prop_seize_basis_datasd.csv')
df10 = pd.read_csv('datasd_ripa/ripa_prop_seize_type_datasd.csv')
df11 = pd.read_csv('datasd_ripa/ripa_stop_result_datasd.csv', low_memory=False)


In [25]:
# merge dataframes using `pid` and `stop_id`
df = df1 
for i in range(2, 12):
    df = pd.merge(df, globals()[f'df{i}'], on=['stop_id','pid'], how='outer')
    

In [26]:
na_rows = df[df.isna().all(axis=1)]
df = df.drop(na_rows.index)
len(df)

3053763

In [27]:
df.columns

Index(['stop_id', 'ori', 'agency', 'exp_years', 'date_stop', 'time_stop',
       'stopduration', 'stop_in_response_to_cfs', 'officer_assignment_key',
       'assignment', 'intersection', 'address_block', 'land_mark',
       'address_street', 'highway_exit', 'isschool', 'school_name',
       'address_city', 'beat', 'beat_name', 'pid', 'isstudent',
       'perceived_limited_english', 'perceived_age', 'perceived_gender',
       'gender_nonconforming', 'gend', 'gend_nc', 'perceived_lgbt',
       'reason_for_stop', 'reason_for_stopcode', 'reason_for_stop_code_text',
       'reason_for_stop_detail', 'reason_for_stop_explanation', 'gender',
       'race', 'disability', 'action', 'consented', 'basis_for_search',
       'basis_for_search_explanation', 'contraband', 'basisforpropertyseizure',
       'type_of_property_seized', 'resultkey', 'result', 'code', 'resulttext',
       'unnamed: 0', 'unnamed: 1', 'unnamed: 2', 'unnamed: 3', 'unnamed: 4',
       'unnamed: 5'],
      dtype='object')

In [28]:
# move pid to second column 
df.insert(1, 'pid', df.pop('pid'))

In [29]:
df.head(5)

,stop_id,pid,ori,agency,exp_years,date_stop,time_stop,stopduration,stop_in_response_to_cfs,officer_assignment_key,...,resultkey,result,code,resulttext,unnamed: 0,unnamed: 1,unnamed: 2,unnamed: 3,unnamed: 4,unnamed: 5
0,2443.0,1.0,CA0371100,SD,10.0,2018-07-01,00:01:37,30.0,0.0,1.0,...,6.0,Custodial Arrest without warrant,64005.0,647(F) PC - DISORD CONDUCT:ALCOHOL (M) 64005,NaN,NaN,NaN,NaN,NaN,NaN
1,2444.0,1.0,CA0371100,SD,18.0,2018-07-01,00:03:34,10.0,0.0,1.0,...,2.0,Warning (verbal or written),54395.0,22349(B) VC - EXC 55MPH SPEED:2 LANE RD (I) 54395,NaN,NaN,NaN,NaN,NaN,NaN
2,2447.0,1.0,CA0371100,SD,1.0,2018-07-01,00:05:43,15.0,1.0,10.0,...,1.0,No Action,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2447.0,2.0,CA0371100,SD,1.0,2018-07-01,00:05:43,15.0,1.0,10.0,...,1.0,No Action,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2448.0,1.0,CA0371100,SD,3.0,2018-07-01,00:19:06,5.0,0.0,1.0,...,1.0,No Action,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# after the merge, there are > 3 million rows
print('Number of observations:', len(df))
print('Total stops:', df['stop_id'].nunique())
print('Total persons stopped:', df[['stop_id','pid']].nunique().sum())

Number of observations: 3053763
Total stops: 645338
Total persons stopped: 645390


In [31]:
# no duplicated rows
df.duplicated().value_counts()

False    3053763
Name: count, dtype: int64

In [32]:
# check for duplicates
df.isna().sum()/len(df)

stop_id                         0.009233
pid                             0.009233
ori                             0.009233
agency                          0.009233
exp_years                       0.009233
date_stop                       0.009233
time_stop                       0.009233
stopduration                    0.009233
stop_in_response_to_cfs         0.009233
officer_assignment_key          0.009233
assignment                      0.009233
intersection                    0.940056
address_block                   0.069906
land_mark                       0.999965
address_street                  0.033840
highway_exit                    0.994086
isschool                        0.009233
school_name                     0.998933
address_city                    0.009234
beat                            0.009234
beat_name                       0.009234
isstudent                       0.009233
perceived_limited_english       0.009233
perceived_age                   0.009233
perceived_gender

**Notes:**
* `consented` field indicates the response by the person(s) stopped if the officer asked for consent to search (Yes/N/A) is 96% blank. Indicates that officers did not for consent?
    
* most did not have a perceived `disabiility` (96%)
* what is `gender_nc` ( 2365 values of 5, rest N/A)

In [33]:
# drop columns with > 80% missing 
columns_to_drop = ['ori', 'agency', 'intersection', 'highway_exit', 'school_name', 'gend_nc','consented', 'unnamed: 0', 'unnamed: 1',
                    'unnamed: 2', 'unnamed: 3', 'unnamed: 4', 'unnamed: 5']
df = df.drop(columns = columns_to_drop, axis=1)

Reasons to drop columns: 
* `ori` = CA0371100 for all observations 
* `agency` = SD for all observations
* all others due to high number of null observations (>80%)

In [37]:
# export merged data to csv
df.to_csv('sd_ripa_reports.csv', index=False)